In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.factorplots import interaction_plot

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#create Spark session
spark = SparkSession.builder.appName('Yelpreview').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '32g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '16'), ('spark.cores.max', '16'), ('spark.driver.memory','32g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'application_1547750003855_6562'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.yarn.appMasterEnv.MKL_NUM_THREADS', '1'),
 ('spark.driver.memory', '32g'),
 ('spark.executor.instances', '32'),
 ('spark.driver.appUIAddress', 'http://md01.rcc.local:4041'),
 ('spark.sql.queryExecutionListeners',
  'com.cloudera.spark.lineage.NavigatorQueryListener'),
 ('spark.cores.max', '16'),
 ('spark.ui.killEnabled', 'true'),
 ('spark.lineage.log.dir', '/var/log/spark/lineage'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'md01.rcc.local,md02.rcc.local'),
 ('spark.eventLog.dir', 'hdfs://nameservice1/user/spark/applicationHistory'),
 ('spark.yarn.am.extraLibraryPath',
  '/opt/cloudera/parcels/CDH-6.1.0-1.cdh6.1.0.p0.770702/lib/hadoop/lib/native'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.executor.extraLibraryPath',
  '/opt/cloudera/parcels/CDH-6.1.0-1.cdh6.1.0.p0.770702/lib/hadoop/lib/native'),
 ('spark.

In [3]:
!ls /home/desen/data/

business.json		      food-inspections.csv  user.json
Business_Licenses.csv	      LookUp.csv	    winemag-data-130k-v2.csv
checkin.json		      output_q8.csv	    winemag-data_first150k.csv
Crimes_-_2001_to_present.csv  review.json
err_q8.txt		      tip.json


In [4]:
# hdfs dfs put
review = spark.read.json("/user/jezhang/data/review.json")

In [5]:
#review.createOrReplaceTempView("review")
print(review.printSchema())
review.show(5)

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

None
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|ujmEBvifdJM6h6RLv...|   0|2013-05-07 04:34:36|    1|Q1sbwvVQXV2734tPg...|  1.0|Total bill for th...|     6|hG7b0MtEbXx5QzbzE...|
|NZnhc2sEQy3RmzKTZ...|   0|2017-01-14 21:30:33|    0|GJXCdrto3ASJOqKeV...|  5.0|I *adore* Travis ...|     0|yXQM5uF2jS6es16SJ...|
|W

In [6]:
business = spark.read.json("/user/jezhang/data/business.json")

In [7]:
#business.createOrReplaceTempView("business")
print(business.printSchema())
business.show(5)

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

### Queries

In [8]:
business_city = business.select('business_id', 'city')
business_city.show(12)

+--------------------+-------------+
|         business_id|         city|
+--------------------+-------------+
|1SWheh84yJXfytovI...|      Phoenix|
|QXAEGFB4oINsVuTFx...|  Mississauga|
|gnKjwL_1w79qoiV3I...|    Charlotte|
|xvX2CttrVhyG2z1dF...|     Goodyear|
|HhyxOkGAM07SRYtlQ...|    Charlotte|
|68dUKd8_8liJ7in4a...|  Mississauga|
|5JucpCfHZltJh5r1J...|      Calgary|
|gbQN7vr_caG_A1ugS...|    Las Vegas|
|Y6iyemLX_oylRpnr3...|     Glendale|
|4GBVPIYRvzGh4K4Tk...|Fairview Park|
|fcXOEZdXYeZqnQ3lG...|      Calgary|
|1Dfx3zM-rW4n-31Ke...|      Phoenix|
+--------------------+-------------+
only showing top 12 rows



In [9]:
review_business_user_city = review.select('business_id', 'user_id')
review_business_user_city.show(12)

+--------------------+--------------------+
|         business_id|             user_id|
+--------------------+--------------------+
|ujmEBvifdJM6h6RLv...|hG7b0MtEbXx5QzbzE...|
|NZnhc2sEQy3RmzKTZ...|yXQM5uF2jS6es16SJ...|
|WTqjgwHlXbSFevF32...|n6-Gk65cPZL6Uz8qR...|
|ikCg8xy5JIg_NGPx-...|dacAIZ6fTM6mqwW5u...|
|b1b1eb3uo-w561D0Z...|ssoyf2_x0EQMed6fg...|
|eU_713ec6fTGNO4Be...|w31MKYsNFMrjhWxxA...|
|3fw2X5bZYeW9xCz_z...|jlu4CztcSxrKx56ba...|
|zvO-PJCpNk4fgAVUn...|d6xvYpyzcfbF_AZ8v...|
|b2jN2mm9Wf3RcrZCg...|sG_h0dIzTKWa3Q6fm...|
|oxwGyA17NL6c5t1Et...|nMeCE5-xsdleyxYuN...|
|8mIrX_LrOnAqWsB5J...|FIk4lQQu1eTe2EpzQ...|
|mRUVMJkUGxrByzMQ2...|-mA3-1mN4JIEkqOtd...|
+--------------------+--------------------+
only showing top 12 rows



In [10]:
user_city_spark = review_business_user_city.join(business_city, ["business_id"], 'left')
user_city_spark.show(12)

+--------------------+--------------------+----------+
|         business_id|             user_id|      city|
+--------------------+--------------------+----------+
|ujmEBvifdJM6h6RLv...|hG7b0MtEbXx5QzbzE...| Las Vegas|
|NZnhc2sEQy3RmzKTZ...|yXQM5uF2jS6es16SJ...| Las Vegas|
|WTqjgwHlXbSFevF32...|n6-Gk65cPZL6Uz8qR...|  Chandler|
|ikCg8xy5JIg_NGPx-...|dacAIZ6fTM6mqwW5u...|   Calgary|
|b1b1eb3uo-w561D0Z...|ssoyf2_x0EQMed6fg...|Scottsdale|
|eU_713ec6fTGNO4Be...|w31MKYsNFMrjhWxxA...|Pittsburgh|
|3fw2X5bZYeW9xCz_z...|jlu4CztcSxrKx56ba...|   Markham|
|zvO-PJCpNk4fgAVUn...|d6xvYpyzcfbF_AZ8v...|Scottsdale|
|b2jN2mm9Wf3RcrZCg...|sG_h0dIzTKWa3Q6fm...| Cleveland|
|oxwGyA17NL6c5t1Et...|nMeCE5-xsdleyxYuN...| Las Vegas|
|8mIrX_LrOnAqWsB5J...|FIk4lQQu1eTe2EpzQ...| Las Vegas|
|mRUVMJkUGxrByzMQ2...|-mA3-1mN4JIEkqOtd...|      Mesa|
+--------------------+--------------------+----------+
only showing top 12 rows



In [11]:
user_city_df = user_city_spark.groupby('user_id', 'city').count().toPandas()
user_city_df.head(12)

,user_id,city,count
0,yXQM5uF2jS6es16SJzNHfg,Las Vegas,1
1,ocC1kDdcWg6hVaTDJssM-Q,Las Vegas,58
2,Pow-A3nsUlagunpPccU6yQ,Las Vegas,5
3,ps87csBgK6zDWeDjFCrjZg,Toronto,19
4,cAxlIVBNys8MkHjUqDibAw,Las Vegas,6
5,9-3lHX4tvYHcRmXJAwhPpA,Las Vegas,6
6,EiP1OFgs-XGcKZux0OKWIA,Toronto,128
7,XqIJKZ1xgLyspleE9vN2Rw,Scottsdale,3
8,WxlXbcB-2UmwzYpkjB7cNQ,Las Vegas,1
9,yZiRD63JCburnjHblf_P3w,Phoenix,1


### looking at user, trying to figure out who is living where -- Using Pandas

In [12]:
user_city_group = user_city_df.groupby(['user_id', 'city'])
user_city = 100 * user_city_group.sum() / user_city_df[['user_id', 'count']].groupby('user_id').sum()
user_city.reset_index(inplace = True)

Threshold = 70
user_city['local'] = user_city['count'].apply(lambda x: True if x >= Threshold else False)

user_city.head()

,user_id,city,count,local
0,---1lKK3aKOuomHnwAkAow,Henderson,6.25000,False
1,---1lKK3aKOuomHnwAkAow,Las Vegas,91.40625,True
2,---1lKK3aKOuomHnwAkAow,North Las Vegas,2.34375,False
3,---89pEy_h9PvHwcHNbpyg,Las Vegas,100.00000,True
4,---94vtJ_5o_nikEs6hUjg,Phoenix,100.00000,True


In [13]:
user_charlotte = user_city[user_city["city"].str.contains("Charlotte", case = False)]
user_charlotte.head(10)

,user_id,city,count,local
11,--0RtXvcOIE4XbErYca6Rw,Charlotte,100.0,True
26,--2DMAd_kGHgbF88H2SLuw,Charlotte,100.0,True
43,--3WaS23LcIXtxyFULJHTA,Charlotte,20.0,False
55,--44NNdtngXMzsxyN7ju6Q,Charlotte,40.0,False
70,--5-MQZPsn2CJkyMFQkOgQ,Charlotte,100.0,True
82,--6D_IuxyKTN53pHi904ag,Charlotte,50.0,False
84,--6Ke7_lBBM6XAramtPoWw,Charlotte,100.0,True
95,--8AvHHmifvY0oybU_G86Q,Charlotte,100.0,True
100,--91OWkE0s-7QAY8g_DKiw,Charlotte,100.0,True
112,--B6JIfTRS2gBmqKhCaI5g,Charlotte,100.0,True


### looking at city, trying to figure out which city is a  tourist destination

In [14]:
import pandas as pd
user_city_cross = pd.crosstab(user_city.city, user_city.local).apply(lambda r: 100 * r / r.sum(), axis=1)
user_city_cross.reset_index(inplace = True)
user_city_cross.head(20)

local,city,False,True
0,,66.666667,33.333333
1,110 Las Vegas,86.764706,13.235294
2,AGINCOURT,85.714286,14.285714
3,AVON,80.000000,20.000000
4,AZ,100.000000,0.000000
5,Agincourt,80.000000,20.000000
6,Ahwahtukee,88.235294,11.764706
7,Ahwatukee,91.149542,8.850458
8,Ahwatukee Foothills Village,100.000000,0.000000
9,Airdrie,55.259467,44.740533


In [15]:
user_city_cross[user_city_cross["city"].str.contains('vegas', case = False)]

local,city,False,True
1,110 Las Vegas,86.764706,13.235294
122,C Las Vegas,95.652174,4.347826
374,Henderson and Las vegas,100.000000,0.000000
430,LAS VEGAS,88.459016,11.540984
434,La Vegas,42.857143,57.142857
442,Lake Las Vegas,100.000000,0.000000
450,Las Vegas,80.000000,20.000000
451,Las Vegas,15.218186,84.781814
452,Las Vegas,88.181818,11.818182
453,Las Vegas & Henderson,85.000000,15.000000


### Using PySpark

In [16]:
user_city_tot = user_city_spark.groupby('user_id', 'city').count()
print(user_city_tot.show(12))

user_city_tot.registerTempTable("user_city_tot")

+--------------------+----------+-----+
|             user_id|      city|count|
+--------------------+----------+-----+
|yXQM5uF2jS6es16SJ...| Las Vegas|    1|
|ocC1kDdcWg6hVaTDJ...| Las Vegas|   58|
|Pow-A3nsUlagunpPc...| Las Vegas|    5|
|ps87csBgK6zDWeDjF...|   Toronto|   19|
|cAxlIVBNys8MkHjUq...| Las Vegas|    6|
|9-3lHX4tvYHcRmXJA...| Las Vegas|    6|
|EiP1OFgs-XGcKZux0...|   Toronto|  128|
|XqIJKZ1xgLyspleE9...|Scottsdale|    3|
|WxlXbcB-2UmwzYpkj...| Las Vegas|    1|
|yZiRD63JCburnjHbl...|   Phoenix|    1|
|teXLmGR-kiEEVW93T...| Las Vegas|    5|
|9nRbPCjtfv7WQjbQF...| Las Vegas|   11|
+--------------------+----------+-----+
only showing top 12 rows

None


In [17]:
user_city_percent = sqlContext.sql("""
  WITH t1 AS 
   (SELECT user_id, city, Count(*) AS n 
    FROM user_city_tot
    GROUP BY user_id, city)
  SELECT user_id, city, 
    100 * (0.0+n)/(COUNT(*) OVER (PARTITION BY user_id)) AS percent
  FROM t1""")

user_city_percent.show(12)

+--------------------+----------+-----------------+
|             user_id|      city|          percent|
+--------------------+----------+-----------------+
|--CJT4d-S8UhwqHe0...|Scottsdale| 50.0000000000000|
|--CJT4d-S8UhwqHe0...|   Phoenix| 50.0000000000000|
|-0Ji0nOyFe-4yo8BK...|  Matthews|100.0000000000000|
|-0XPr1ilUAfp-yIXZ...|Pittsburgh|100.0000000000000|
|-1KKYzibGPyUX-Mwk...|   Madison|100.0000000000000|
|-1zQA2f_syMAdA04P...| Las Vegas|100.0000000000000|
|-2Pb5d2WBPtbyGT_b...| Las Vegas|100.0000000000000|
|-2mPrKWc9UYdvTrOZ...|Lake Wylie|100.0000000000000|
|-3bsS2i9xqjNnIA1f...| Las Vegas|100.0000000000000|
|-3i9bhfvrM3F1wsC9...|   Phoenix| 33.3333333333333|
|-3i9bhfvrM3F1wsC9...| Las Vegas| 33.3333333333333|
|-3i9bhfvrM3F1wsC9...| Henderson| 33.3333333333333|
+--------------------+----------+-----------------+
only showing top 12 rows



In [18]:
from pyspark.sql.functions import col, when
user_city_lookup = user_city_percent.withColumn('local', (when(col("percent") >= 70, True).otherwise(False)))

In [19]:
user_city_lookup.show(20)

+--------------------+-----------+-----------------+-----+
|             user_id|       city|          percent|local|
+--------------------+-----------+-----------------+-----+
|--CJT4d-S8UhwqHe0...| Scottsdale| 50.0000000000000|false|
|--CJT4d-S8UhwqHe0...|    Phoenix| 50.0000000000000|false|
|-0Ji0nOyFe-4yo8BK...|   Matthews|100.0000000000000| true|
|-0XPr1ilUAfp-yIXZ...| Pittsburgh|100.0000000000000| true|
|-1KKYzibGPyUX-Mwk...|    Madison|100.0000000000000| true|
|-1zQA2f_syMAdA04P...|  Las Vegas|100.0000000000000| true|
|-2Pb5d2WBPtbyGT_b...|  Las Vegas|100.0000000000000| true|
|-2mPrKWc9UYdvTrOZ...| Lake Wylie|100.0000000000000| true|
|-3bsS2i9xqjNnIA1f...|  Las Vegas|100.0000000000000| true|
|-3i9bhfvrM3F1wsC9...|    Phoenix| 33.3333333333333|false|
|-3i9bhfvrM3F1wsC9...|  Las Vegas| 33.3333333333333|false|
|-3i9bhfvrM3F1wsC9...|  Henderson| 33.3333333333333|false|
|-47g7LR58tpHlm7Bm...|  Las Vegas|100.0000000000000| true|
|-4Anvj46CWf57KWI9...|    Toronto|100.0000000000000| tru

### User Over Time

In [20]:
user = spark.read.json("/user/jezhang/data/user.json")

In [21]:
#business.createOrReplaceTempView("business")
print(user.printSchema())
user.show(5)

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)

None
+-------------+---------------+---------------+----------------+

In [22]:
from pyspark.sql.functions import countDistinct

user.agg(countDistinct('user_id').alias('Total Numer of User Todate')).show()

user.count()

+--------------------------+
|Total Numer of User Todate|
+--------------------------+
|                   1637138|
+--------------------------+



1637138

### User Growth Over Year

In [23]:
from pyspark.sql.functions import trunc

user_year = user.groupby(trunc('yelping_since', 'year').alias('year')).count().orderBy('year', ascending=False)

user_year.show(20)

+----------+------+
|      year| count|
+----------+------+
|2018-01-01| 78016|
|2017-01-01|120531|
|2016-01-01|199148|
|2015-01-01|238660|
|2014-01-01|225437|
|2013-01-01|197220|
|2012-01-01|182900|
|2011-01-01|168467|
|2010-01-01|106840|
|2009-01-01| 63977|
|2008-01-01| 32544|
|2007-01-01| 16480|
|2006-01-01|  5836|
|2005-01-01|  1001|
|2004-01-01|    81|
+----------+------+



### Year's user over number of reviews and average star

In [24]:
review.agg(countDistinct('review_id').alias('Total Numer of Review Todate')).show()

review.count()

+----------------------------+
|Total Numer of Review Todate|
+----------------------------+
|                     6685900|
+----------------------------+



6685900

In [25]:
user_year_join = user.select('user_id', 'yelping_since')
    
review_year_star = review.select('review_id', 'user_id', 'stars')
user_review_year = review_year_star.join(user_year_join, ["user_id"], 'left')
user_review_year.show(12)

+--------------------+--------------------+-----+-------------------+
|             user_id|           review_id|stars|      yelping_since|
+--------------------+--------------------+-----+-------------------+
|--CJT4d-S8UhwqHe0...|bklQqMXHT-JYiVQdB...|  1.0|2015-08-18 19:10:59|
|--CJT4d-S8UhwqHe0...|5aq0aGO3LNcAIV2Rl...|  4.0|2015-08-18 19:10:59|
|-0Ji0nOyFe-4yo8BK...|a0r23pA6D4HDLb9P5...|  4.0|2009-05-15 19:50:34|
|-0XPr1ilUAfp-yIXZ...|DfAjJ3KCimOdqplv4...|  2.0|2007-12-26 17:32:44|
|-1KKYzibGPyUX-Mwk...|TbU3S2kWmfajlVov3...|  4.0|2009-08-07 13:42:19|
|-1zQA2f_syMAdA04P...|AF7_icUaLQVgj7d29...|  1.0|2014-06-22 02:44:41|
|-1zQA2f_syMAdA04P...|d1vPdEGTP75fS_bd2...|  5.0|2014-06-22 02:44:41|
|-1zQA2f_syMAdA04P...|_aY8ReBavQVGHhCh7...|  1.0|2014-06-22 02:44:41|
|-1zQA2f_syMAdA04P...|y0BLDiU8nQi22t9OF...|  5.0|2014-06-22 02:44:41|
|-1zQA2f_syMAdA04P...|08j8zUlyJylKcJa_r...|  5.0|2014-06-22 02:44:41|
|-1zQA2f_syMAdA04P...|QCQJC9kbCkhzTRosS...|  1.0|2014-06-22 02:44:41|
|-2Pb5d2WBPtbyGT_b..

In [26]:
review_year = user_review_year.groupby(trunc('yelping_since', 'year').alias('year')).count().orderBy('year', ascending=False)

review_year.show(20)

+----------+------+
|      year| count|
+----------+------+
|2018-01-01|141422|
|2017-01-01|295846|
|2016-01-01|546668|
|2015-01-01|772595|
|2014-01-01|853090|
|2013-01-01|831904|
|2012-01-01|849138|
|2011-01-01|877084|
|2010-01-01|654385|
|2009-01-01|435993|
|2008-01-01|245372|
|2007-01-01|127319|
|2006-01-01| 45376|
|2005-01-01|  8180|
|2004-01-01|  1528|
+----------+------+



In [27]:
from pyspark.sql.functions import mean

review_year_star = user_review_year.groupby(trunc('yelping_since', 'year').alias('year')).agg(mean(col('stars')).alias('avg_stars')).orderBy('year', ascending=False)

review_year_star.show(20)

+----------+------------------+
|      year|         avg_stars|
+----------+------------------+
|2018-01-01| 3.603901797457256|
|2017-01-01|3.6824462727229705|
|2016-01-01|3.6988537832834556|
|2015-01-01|3.7066936752114628|
|2014-01-01| 3.725242354265083|
|2013-01-01|3.7275130303496558|
|2012-01-01| 3.722898986972671|
|2011-01-01| 3.727376169215263|
|2010-01-01| 3.730906117958083|
|2009-01-01|3.7008185911241696|
|2008-01-01| 3.733853088371941|
|2007-01-01|3.7303937354204795|
|2006-01-01| 3.733383286318759|
|2005-01-01|3.8117359413202934|
|2004-01-01| 3.360602094240838|
+----------+------------------+



In [28]:
star_city = review.groupby(trunc('date', 'year').alias('year')).agg(mean(col('stars')).alias('avg_stars')).orderBy('year', ascending=False)

In [29]:
star_city.show(20)

+----------+------------------+
|      year|         avg_stars|
+----------+------------------+
|2018-01-01|3.7640443522844413|
|2017-01-01| 3.757166332833199|
|2016-01-01|3.7352150464239626|
|2015-01-01|3.6983137337253256|
|2014-01-01| 3.675515207232054|
|2013-01-01| 3.651219025675054|
|2012-01-01|3.6407066386989566|
|2011-01-01|3.6662198258641583|
|2010-01-01| 3.686461707589107|
|2009-01-01|3.6812094135787214|
|2008-01-01|3.7111269987967983|
|2007-01-01|3.8117256533732293|
|2006-01-01|3.8027947254477463|
|2005-01-01| 4.026255707762557|
|2004-01-01| 4.461538461538462|
+----------+------------------+



In [30]:
user_year_join = user.select('user_id', 'yelping_since')
    
review_year_star = review.select('review_id', 'user_id', 'stars', 'date')
user_review_year = review_year_star.join(user_year_join, ["user_id"], 'left')

In [31]:
user_review_year = user_review_year.withColumn('year_since', trunc(col('yelping_since'), 'year'))
user_review_year = user_review_year.withColumn('year_date', trunc(col('date'), 'year'))

user_review_year.show(12)

+--------------------+--------------------+-----+-------------------+-------------------+----------+----------+
|             user_id|           review_id|stars|               date|      yelping_since|year_since| year_date|
+--------------------+--------------------+-----+-------------------+-------------------+----------+----------+
|--CJT4d-S8UhwqHe0...|bklQqMXHT-JYiVQdB...|  1.0|2015-08-18 19:11:04|2015-08-18 19:10:59|2015-01-01|2015-01-01|
|--CJT4d-S8UhwqHe0...|5aq0aGO3LNcAIV2Rl...|  4.0|2015-08-18 19:22:17|2015-08-18 19:10:59|2015-01-01|2015-01-01|
|-0Ji0nOyFe-4yo8BK...|a0r23pA6D4HDLb9P5...|  4.0|2014-03-21 17:09:59|2009-05-15 19:50:34|2009-01-01|2014-01-01|
|-0XPr1ilUAfp-yIXZ...|DfAjJ3KCimOdqplv4...|  2.0|2012-07-06 03:43:34|2007-12-26 17:32:44|2007-01-01|2012-01-01|
|-1KKYzibGPyUX-Mwk...|TbU3S2kWmfajlVov3...|  4.0|2014-09-19 01:12:10|2009-08-07 13:42:19|2009-01-01|2014-01-01|
|-1zQA2f_syMAdA04P...|08j8zUlyJylKcJa_r...|  5.0|2016-07-29 19:44:01|2014-06-22 02:44:41|2014-01-01|2016

In [32]:
star_city = user_review_year.filter(col('year_since') == col('year_date')).groupby('year_since').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_since', ascending=False)
star_city.show(20)

+----------+------------------+
|year_since|         avg_stars|
+----------+------------------+
|2018-01-01| 3.603901797457256|
|2017-01-01|3.6530376600567607|
|2016-01-01|3.6564239376607426|
|2015-01-01| 3.630105111487983|
|2014-01-01|3.6234199599354215|
|2013-01-01|3.5959762203234713|
|2012-01-01|3.5872975034228007|
|2011-01-01| 3.643065615490025|
|2010-01-01| 3.694322715276499|
|2009-01-01| 3.674557125508414|
|2008-01-01|3.7267742824761845|
|2007-01-01|3.8419567262464724|
|2006-01-01|3.7992957746478875|
|2005-01-01|4.0231749710312865|
|2004-01-01| 4.461538461538462|
+----------+------------------+



In [33]:
user_review_year.filter(user_review_year.year_since.like('%2004%')).show(12)

+--------------------+--------------------+-----+-------------------+-------------------+----------+----------+
|             user_id|           review_id|stars|               date|      yelping_since|year_since| year_date|
+--------------------+--------------------+-----+-------------------+-------------------+----------+----------+
|yYSBB5q7bY-qSVvmM...|0O7kc-T1SrZpF84jM...|  5.0|2004-10-19 03:05:42|2004-10-12 20:18:03|2004-01-01|2004-01-01|
|z2JQPh4S_eeq04Ym7...|q-C-lrBnIJ7qoD4Cz...|  3.0|2017-01-20 17:34:35|2004-10-15 18:28:30|2004-01-01|2017-01-01|
|zXHee8sM-qLALzPi9...|rsCMS13UzmQ5pF1Rg...|  2.0|2010-04-08 13:31:25|2004-10-15 21:12:19|2004-01-01|2010-01-01|
|zXHee8sM-qLALzPi9...|YoVXwCaVDJVgoscfB...|  1.0|2017-11-29 00:05:01|2004-10-15 21:12:19|2004-01-01|2017-01-01|
|0sidDfoTIHle5vvHE...|ic3WHNrH1cbVsS_2o...|  5.0|2006-01-16 20:55:27|2004-10-12 15:47:20|2004-01-01|2006-01-01|
|0sidDfoTIHle5vvHE...|FfES_M5Tn3VdCwWda...|  5.0|2011-11-11 22:17:20|2004-10-12 15:47:20|2004-01-01|2011

In [35]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2004%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|            3.4375|
|2017-01-01| 3.443298969072165|
|2016-01-01| 3.536842105263158|
|2015-01-01|3.3214285714285716|
|2014-01-01|3.1037735849056602|
|2013-01-01| 3.279279279279279|
|2012-01-01|3.3304347826086955|
|2011-01-01|3.4761904761904763|
|2010-01-01| 3.296511627906977|
|2009-01-01|3.1564625850340136|
|2008-01-01| 3.278409090909091|
|2007-01-01|3.5681818181818183|
|2006-01-01|3.4415584415584415|
|2005-01-01| 4.230769230769231|
|2004-01-01| 4.461538461538462|
+----------+------------------+



In [36]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2005%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01| 3.917197452229299|
|2017-01-01| 3.911764705882353|
|2016-01-01|            3.9625|
|2015-01-01|3.8021739130434784|
|2014-01-01|3.8194444444444446|
|2013-01-01|3.8118811881188117|
|2012-01-01|3.7819148936170213|
|2011-01-01|3.8111888111888113|
|2010-01-01|3.7178002894356004|
|2009-01-01| 3.747183979974969|
|2008-01-01|3.7196870925684484|
|2007-01-01|3.7175438596491226|
|2006-01-01| 3.843385214007782|
|2005-01-01|4.0231749710312865|
+----------+------------------+



In [37]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2006%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01| 3.783178590933916|
|2017-01-01|  3.83857264231096|
|2016-01-01| 3.815040650406504|
|2015-01-01|3.7334710743801653|
|2014-01-01|3.6939728353140917|
|2013-01-01| 3.689890193108671|
|2012-01-01|3.7011813759555245|
|2011-01-01|3.7016604177825387|
|2010-01-01|3.6810431293881645|
|2009-01-01|3.6960091220068416|
|2008-01-01| 3.698005698005698|
|2007-01-01|3.7647979956154085|
|2006-01-01|3.7992957746478875|
+----------+------------------+



In [38]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2007%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|3.8317325452530477|
|2017-01-01|3.8242135082749367|
|2016-01-01| 3.811248942768537|
|2015-01-01|3.7556725586059923|
|2014-01-01|3.7251556662515566|
|2013-01-01| 3.719921298573537|
|2012-01-01| 3.707862818904224|
|2011-01-01| 3.670691216420163|
|2010-01-01|3.6666426512968298|
|2009-01-01| 3.675477776964194|
|2008-01-01|3.6907094594594594|
|2007-01-01|3.8419567262464724|
+----------+------------------+



In [39]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2008%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01| 3.840304761904762|
|2017-01-01| 3.808272081724181|
|2016-01-01|3.7784235810450153|
|2015-01-01|  3.74822695035461|
|2014-01-01| 3.734280639431616|
|2013-01-01|3.7148080438756854|
|2012-01-01|  3.70066851607622|
|2011-01-01|3.7242470496258515|
|2010-01-01| 3.706685236768802|
|2009-01-01|3.6935696837657623|
|2008-01-01|3.7267742824761845|
+----------+------------------+



In [40]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2009%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|3.8163399552690436|
|2017-01-01|3.7881474701827647|
|2016-01-01|3.7641931684334513|
|2015-01-01|3.7240009054098238|
|2014-01-01| 3.697907668682215|
|2013-01-01| 3.661775015189045|
|2012-01-01|3.6430225825130282|
|2011-01-01|3.6553442028985508|
|2010-01-01| 3.670877625430083|
|2009-01-01| 3.674557125508414|
+----------+------------------+



In [41]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2010%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01| 3.842451993556911|
|2017-01-01|3.8241622789664302|
|2016-01-01|3.7874529633113827|
|2015-01-01|3.7445115443332115|
|2014-01-01| 3.706815100850347|
|2013-01-01|3.6894684277897345|
|2012-01-01|3.6654344820032585|
|2011-01-01|3.6804658435326467|
|2010-01-01| 3.694322715276499|
|2009-01-01|               4.0|
|2008-01-01|               3.6|
+----------+------------------+



In [42]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2011%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|3.8212704473915062|
|2017-01-01|3.8167600126809678|
|2016-01-01|3.7803333333333335|
|2015-01-01|3.7349405698230975|
|2014-01-01|3.6927796676581117|
|2013-01-01|3.6752070019182015|
|2012-01-01| 3.656992585419574|
|2011-01-01| 3.643065615490025|
|2010-01-01|               5.0|
|2009-01-01|               4.0|
+----------+------------------+



In [43]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2012%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|3.8266005322582073|
|2017-01-01|3.8031460467539633|
|2016-01-01|3.7671076483974235|
|2015-01-01| 3.726133571588117|
|2014-01-01| 3.681100443491398|
|2013-01-01|3.6456830102622577|
|2012-01-01|3.5872975034228007|
|2011-01-01|               3.5|
|2010-01-01|               4.0|
|2009-01-01|               5.0|
|2008-01-01|               4.0|
+----------+------------------+



In [44]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2013%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|3.8306067961165047|
|2017-01-01|3.7857933630400487|
|2016-01-01|3.7532054352139466|
|2015-01-01| 3.707949401875202|
|2014-01-01| 3.691359713214175|
|2013-01-01|3.5959762203234713|
|2012-01-01| 4.372549019607843|
|2011-01-01| 4.264367816091954|
|2010-01-01|              4.25|
|2009-01-01|4.2727272727272725|
|2008-01-01|3.6666666666666665|
+----------+------------------+



In [45]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2014%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01| 3.797377708470125|
|2017-01-01|3.7802883299008343|
|2016-01-01|3.7501446503005185|
|2015-01-01|3.6972108094025953|
|2014-01-01|3.6234199599354215|
|2013-01-01|              3.75|
|2012-01-01|3.4166666666666665|
|2011-01-01| 4.027777777777778|
|2010-01-01|               5.0|
+----------+------------------+



In [46]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2015%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|3.7643115102120497|
|2017-01-01|3.7399126394747677|
|2016-01-01| 3.714046350129199|
|2015-01-01| 3.630105111487983|
|2008-01-01|3.7777777777777777|
+----------+------------------+



In [47]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2016%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|3.7346112886048988|
|2017-01-01|3.7168689485885724|
|2016-01-01|3.6564239376607426|
+----------+------------------+



In [48]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2017%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+------------------+
| year_date|         avg_stars|
+----------+------------------+
|2018-01-01|3.7193038577249706|
|2017-01-01|3.6530376600567607|
+----------+------------------+



In [49]:
star_city = user_review_year.filter(user_review_year.year_since.like('%2018%')).groupby('year_date').agg(mean(col('stars')).alias('avg_stars')).orderBy('year_date', ascending=False)
star_city.show(20)

+----------+-----------------+
| year_date|        avg_stars|
+----------+-----------------+
|2018-01-01|3.603901797457256|
+----------+-----------------+

